In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Import Library
import pandas as pd
import numpy as np
import folium
import os
import branca
import branca.colormap as cmp

from datetime import datetime

# Selecting the Data File
datafile = "cleaned_dataset_killed.csv"

In [3]:
# Initializing Dataframe
df = pd.read_csv(datafile)
df

,age,race,date,state,police_agency,description of death,was armed,weapon on victim,geography type,male,shooting_involved,taser_involved,physical_violence_involved,justified,armed_with_gun,armed_with_knife,was_a_threat,was_fleeing,mental_illness
0,66.0,Hispanic,2020-11-18 00:00:00,TX,Tyler Junior College Police Department,Martinez allegedly approached a Tyler Junior C...,Allegedly Armed,unknown weapon,NaN,1.0,1,0,0,NaN,0.0,0.0,0.0,0.0,0.0
1,28.0,White,2020-11-17 00:00:00,FL,"St. Augustine Police Department, St. Johns Cou...",Police were called to a home where a man was a...,Allegedly Armed,gun,NaN,1.0,1,0,0,NaN,1.0,0.0,1.0,NaN,0.0
2,35.0,White,2020-11-14 00:00:00,NH,Meredith Police Department,David Donovan was shot by a Meredith police of...,Unclear,undetermined,Rural,1.0,1,0,0,NaN,0.0,0.0,NaN,NaN,0.0
3,38.0,Hispanic,2020-11-14 00:00:00,CA,Sacramento Police Department,"Around 8 p.m., officers responded to reports o...",Allegedly Armed,gun,Urban,1.0,1,0,0,NaN,1.0,0.0,NaN,NaN,NaN
4,40.0,White,2020-11-14 00:00:00,FL,Escambia County Sheriff's Office,Deputies were serving a warrant to Jake Settle...,Vehicle,vehicle,Suburban,1.0,1,0,0,NaN,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9719,NaN,NaN,2020-11-27,OR,NaN,NaN,gun,NaN,NaN,1.0,1,0,0,NaN,1.0,NaN,0.0,NaN,NaN
9720,43.0,Black,2020-11-29,LA,NaN,NaN,gun,NaN,NaN,1.0,1,0,0,NaN,1.0,NaN,1.0,0.0,NaN
9721,NaN,NaN,2020-11-30,AZ,NaN,NaN,BB gun,NaN,NaN,1.0,1,0,0,NaN,1.0,NaN,1.0,0.0,NaN
9722,38.0,Hispanic,2020-11-30,TX,NaN,NaN,knife,NaN,NaN,1.0,1,0,0,NaN,NaN,1.0,0.0,0.0,NaN


In [4]:
# Creating Year Columns
df['date'] = pd.to_datetime(df['date'])
df['year'] = df['date'].apply(lambda x: x.year)

In [5]:
# Counting shoootings for each states
new_df =  df["state"].value_counts()
new_df = new_df.to_frame()
new_df = new_df.reset_index()
new_df = new_df.rename(columns={"index": "State", "state": "Number"})
new_df

,State,Number
0,CA,1475
1,TX,900
2,FL,695
3,AZ,415
4,GA,324
5,CO,305
6,OK,288
7,WA,282
8,NC,277
9,OH,266


In [6]:
# Step colors for the map
step = cmp.StepColormap(
 ['#ffffcc','#ffeda0','#fed976','#feb24c','#fd8d3c','#fc4e2a','#e31a1c','#b10026'],
 vmin=0, vmax=1300,
 index=[0,40,90,150,220,300,500,700,1300],
 caption='Numbers of Shootings' 
)

In [7]:
# Generating folium file
state_geo = 'us-states.json'
m = folium.Map([48,-102], tiles='cartodbpositron', zoom_start=4)
shooting_dict = new_df.set_index('State')['Number']
for i in list(range(8)):
    
    folium.GeoJson(
        state_geo,
        style_function=lambda feature: {
            'fillColor': step(shooting_dict[feature['id']]),
            'color': 'black',     #border color for the color fills
            'weight': 1,          #how thick the border has to be
            'dashArray': '4, 3'  #dashed lines length,space between them
        }
    ).add_to(m)
    step.add_to(m)

In [8]:
incomefile = "nst-est2019-01.xlsx"
# Data Cleaning
df_pop = pd.read_excel(incomefile)
df_state = df_pop[8:59]
df_state.rename(columns=df_pop.iloc[2],inplace=True)
df_state.columns.values[0] = "State"
df_state = df_state.reset_index()
df_state.drop(columns=['Census', 'Estimates Base','index'],inplace=True)
df_state.columns = df_state.columns.astype(str)

# Dictionary for changing full state name to two-letter abbreviation
us_state_abbrev = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'American Samoa': 'AS',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'District of Columbia': 'DC',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Guam': 'GU',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Northern Mariana Islands':'MP',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Pennsylvania': 'PA',
    'Puerto Rico': 'PR',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virgin Islands': 'VI',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY'
}

In [9]:
def state_modify(x):
    string = x.replace('.','')
    return string

In [10]:
# Modifying State Name
df_state['State'] = df_state['State'].apply(state_modify)
df_state['State'] = df_state['State'].map(us_state_abbrev)

In [11]:
# Prepare dataframe to merge
df_merge = df.groupby(['state','year']).size()
df_merge = df_merge.to_frame().reset_index().rename(columns={0: "count"})
df_merge

,state,year,count
0,AK,2013,1
1,AK,2014,2
2,AK,2015,5
3,AK,2016,9
4,AK,2017,9
...,...,...,...
399,WY,2016,3
400,WY,2017,1
401,WY,2018,5
402,WY,2019,3


In [12]:
# Merging number of shootings dataframe with population dataframe
df_perpare = pd.melt(df_state,id_vars = ['State'],value_vars = df_state.columns[1:])
df_perpare["variable"] = df_perpare["variable"].astype(float)
df_perpare["variable"] = df_perpare["variable"].astype(int)
df_plot = df_perpare.merge(df_merge,how="left",right_on=['state','year'],left_on=['State','variable'])
df_plot['result'] = df_plot['count']/df_plot.value*1000000
df_plot

,State,variable,value,state,year,count,result
0,AL,2010,4785437,NaN,NaN,NaN,NaN
1,AK,2010,713910,NaN,NaN,NaN,NaN
2,AZ,2010,6407172,NaN,NaN,NaN,NaN
3,AR,2010,2921964,NaN,NaN,NaN,NaN
4,CA,2010,37319502,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
505,VA,2019,8.53552e+06,VA,2019.0,13.0,1.52305
506,WA,2019,7.61489e+06,WA,2019.0,48.0,6.30344
507,WV,2019,1.79215e+06,WV,2019.0,17.0,9.48583
508,WI,2019,5.82243e+06,WI,2019.0,22.0,3.77849


In [13]:
# Take out the values for plotting
plot = df_plot[df_plot['result'].notnull()]
plot = plot.reset_index()
plot.drop(columns=['value','state','year','count','index'],inplace=True)
plot.rename(columns={'variable':'Year','result':'Number'},inplace=True)
plot = plot.groupby(['State','Number']).sum()
plot = plot.index.to_list()
state_list = []
sum_list = []


for i in plot:
    if i[0] not in state_list:
        state_list.append(i[0])
    
i = 0
for j in range(len(state_list)):
    sum_ = 0
    count = 0
    while plot[i][0] == state_list[j]:
        sum_ += plot[i][1]
        i += 1
        if i == len(plot):
            break
        count += 1
    sum_list.append(sum_/count)

df_plot = pd.DataFrame({'State':state_list,'Count':sum_list})
df_plot.sort_values(by=['Count'],ascending=False)

,State,Count
32,NM,10.925107
36,OK,9.256031
0,AK,8.527153
3,AZ,7.513456
33,NV,6.792816
49,WV,6.661602
50,WY,6.585744
5,CO,6.460356
26,MT,6.161269
7,DC,5.720501


In [14]:
# Step colors for the map
linear = cmp.LinearColormap(
    ['#ffffcc','#ffeda0','#fed976','#feb24c','#fd8d3c','#fc4e2a','#e31a1c','#b10026'],
    vmin = 0, vmax = 10,
    caption = 'Shootings per million people' #Caption for Color scale or Legend
)

In [15]:
# Generating folium file
state_geo = 'us-states.json'
m = folium.Map([48,-102], tiles='cartodbpositron', zoom_start=4)
shooting_dict = df_plot.set_index('State')['Count']
for i in list(range(8)):
    folium.GeoJson(
        state_geo,
        style_function=lambda feature: {
            'fillColor': linear(shooting_dict[feature['id']]),
            'color': 'black',     #border color for the color fills
            'weight': 1,          #how thick the border has to be
            'dashArray': '4, 3'  #dashed lines length,space between them
        }
    ).add_to(m)
    linear.add_to(m)
